# load env

In [11]:
import os
from apikey import apikey, serpapi_key, API_KEY, BASE_URL, DEPLOYMENT_NAME
from langchain.chat_models import AzureChatOpenAI
from langchain.schema import HumanMessage

os.environ['SERPAPI_API_KEY'] = serpapi_key


In [2]:
llm_azure = AzureChatOpenAI(
    openai_api_base=BASE_URL,
    openai_api_version="2023-05-15",
    deployment_name=DEPLOYMENT_NAME,
    openai_api_key=API_KEY,
    openai_api_type = "azure",
    temperature=0
)

In [5]:
llm_azure([HumanMessage(content="Translate this sentence from English to Chinese. I love programming.")])

AIMessage(content='我喜欢编程。', additional_kwargs={}, example=False)

# 自定义Tools

创建Tool 必要组件：
- name (str), is required and must be unique within a set of tools provided to an agent
- description (str), is optional but recommended, as it is used by an agent to determine tool use
- return_direct (bool), defaults to False
- args_schema (Pydantic BaseModel), is optional but recommended, can be used to provide more information (e.g., few-shot examples) or validation for expected parameters.

In [8]:
from langchain import LLMMathChain, SerpAPIWrapper
from langchain.agents import AgentType, initialize_agent
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool, StructuredTool, Tool, tool


## Completely New Tools - String Input and Output

### 方法一：Tool dataclass -- 从工具库中直接引用工具

In [12]:
search = SerpAPIWrapper()
llm_math_chain = LLMMathChain(llm=llm_azure, verbose=True)
tools = [
    Tool.from_function(
        func=search.run,
        name="search",
        description="useful for when you need to answer questions about current events"
    )
]


/Users/matao/opt/anaconda3/envs/langchain/lib/python3.10/site-packages/langchain/chains/llm_math/base.py:50: UserWarning: Directly instantiating an LLMMathChain with an llm is deprecated. Please instantiate with llm_chain argument or using the from_llm class method.
  warnings.warn(


In [17]:
from pydantic import BaseModel, Field

class CalculatorInput(BaseModel):
    question: str = Field()

tools.append(
    Tool.from_function(
        func=llm_math_chain.run,
        name="Calculator",
        description="useful for when you need to answer questions about math",
        args_schema=CalculatorInput
    )
)

In [13]:
tools


[Tool(name='search', description='useful for when you need to answer questions about current events', args_schema=None, return_direct=False, verbose=False, callbacks=None, callback_manager=None, handle_tool_error=False, func=<bound method SerpAPIWrapper.run of SerpAPIWrapper(search_engine=<class 'serpapi.google_search.GoogleSearch'>, params={'engine': 'google', 'google_domain': 'google.com', 'gl': 'us', 'hl': 'en'}, serpapi_api_key='abc98e5936b29895645aabf66c3e65cbc9693d18166c27eba62a0399f75e8562', aiosession=None)>, coroutine=None)]

In [14]:
agent = initialize_agent(
    tools, llm_azure, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

### 方法二：Subclassing the BaseTool class - 从BaseTool 从继承一个子类

可以直接子类化 BaseTool。如果希望对实例变量有更多的控制，或者希望将回调传播到嵌套链或其他工具，那么这很有用。

In [15]:
from typing import Optional, Type

from langchain.callbacks.manager import(
    AsyncCallbackManagerForToolRun,
    CallbackManagerForToolRun
)

class CustomSearchTool(BaseTool):
    name = "custome_search"
    description = "useful for when you need to answer questions about current events"

    def _run(
            self, query: str, run_manager: Optional[CallbackManagerForToolRun] = None
    ) -> str:
        """Use the tool."""
        return search.run(query)
    
    async def _arun(
            self, query: str, run_manager: Optional[AsyncCallbackManagerForToolRun] = None
    ) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("custom_search does not support async")

In [21]:
class CustomCalculatorTool(BaseTool):
    name = "Calculator"
    description = "useful for when you need to answer questions about math"
    args_schema: Type[BaseModel] = CalculatorInput

    def _run(
            self, query: str, run_manager: Optional[CallbackManagerForToolRun] = None
    ) -> str:
        """Use the tool."""
        return llm_math_chain.run(query)
    
    async def _arun(
            self, query: str, run_manager: Optional[AsyncCallbackManagerForToolRun] = None
    ) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("Calculator does not support async")

In [20]:
tools = [CustomSearchTool(), CustomCalculatorTool()]
agent = initialize_agent(
    tools, llm_azure, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

## 使用 tool 装饰器

还是推荐这个方式进行工具创建，非常快捷

In [27]:
from langchain.tools import tool

@tool
def search_api(query: str) -> str:
    """searches the API for the query."""
    return f"Results for query {query}"

search_api

StructuredTool(name='search_api', description='search_api(query: str) -> str - searches the API for the query.', args_schema=<class 'pydantic.main.search_apiSchemaSchema'>, return_direct=False, verbose=False, callbacks=None, callback_manager=None, handle_tool_error=False, func=<function search_api at 0x12372d120>, coroutine=None)

装饰器中传递参数可以重命名工具和 return_direct 参数

In [26]:
@tool("search_re", return_direct=True)
def search_api(query: str) -> str:
    """Searches the API for the query."""
    return "Results"

search_api

StructuredTool(name='search_re', description='search_re(query: str) -> str - Searches the API for the query.', args_schema=<class 'pydantic.main.search_reSchemaSchema'>, return_direct=True, verbose=False, callbacks=None, callback_manager=None, handle_tool_error=False, func=<function search_api at 0x12372e5f0>, coroutine=None)

In [28]:
class SearchInput(BaseModel):
    query: str = Field(description="should be a search query")

@tool("search", return_direct=True, args_schema=SearchInput)
def search_api(query: str) -> str:
    """Searches the API for the query."""
    return "Results"

search_api

StructuredTool(name='search', description='search(query: str) -> str - Searches the API for the query.', args_schema=<class '__main__.SearchInput'>, return_direct=True, verbose=False, callbacks=None, callback_manager=None, handle_tool_error=False, func=<function search_api at 0x12372c5e0>, coroutine=None)

## 自定义结构工具 - Structured Tools

如果函数工具需要更多结构化参数，则可以使用 StructuredTool 类

### StructuredTool dataclass

In [29]:
import requests
from langchain.tools import StructuredTool

def post_message(url: str, body: dict, parameters: Optional[dict]=None) -> str:
    """Sends a POST request to the given url with the given body and parameters."""
    result = requests.post(url, json=body, params=parameters)
    return f"Status:{result.status_code} - {result.text}"

tool = StructuredTool.from_function(post_message)

In [30]:
tool

StructuredTool(name='post_message', description='post_message(url: str, body: dict, parameters: Optional[dict] = None) -> str - Sends a POST request to the given url with the given body and parameters.', args_schema=<class 'pydantic.main.post_messageSchemaSchema'>, return_direct=False, verbose=False, callbacks=None, callback_manager=None, handle_tool_error=False, func=<function post_message at 0x12372eb00>, coroutine=None)

# the priorities among Tools - 确定Tools之间的优先级

In [40]:
tools = [
    Tool(
        name="Search",
        func=search.run,
        description="useful for when you need to answer questions about current events",
    ),
    Tool(
        name="Music Search",
        func=lambda x: "'All I Want For Christmas Is You' by Mariah Carey.",  # Mock Function
        description="A Music search engine. Use this more than the normal search if the question is about Music, like 'who is the singer of yesterday?' or 'what is the most popular song in 2022?'",
        return_direct = True,
    ),
]

agent = initialize_agent(
    tools,
    llm_azure,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

In [39]:
agent.run("what is the most famous song of christmas")





> Entering new AgentExecutor chain...
I should use Music Search for this question since it's about music.
Action: Music Search
Action Input: "most famous christmas song"
Observation: 'All I Want For Christmas Is You' by Mariah Carey.
Thought:I should double check to make sure this is accurate.
Action: Music Search
Action Input: "most popular christmas song of all time"
Observation: 'All I Want For Christmas Is You' by Mariah Carey.
Thought:I now know the final answer.
Final Answer: 'All I Want For Christmas Is You' by Mariah Carey is the most famous Christmas song.

> Finished chain.


"'All I Want For Christmas Is You' by Mariah Carey is the most famous Christmas song."

In [41]:
agent.run("what is the most famous song of christmas")
# cc：这次是设计return_direct=True 返回的结果，由此来看这里就没有Thought及后续，有结果即返回。




> Entering new AgentExecutor chain...
I should use Music Search for this question since it's about music.
Action: Music Search
Action Input: "most famous christmas song"
Observation: 'All I Want For Christmas Is You' by Mariah Carey.


> Finished chain.


"'All I Want For Christmas Is You' by Mariah Carey."

# 用工具直接返回结果

In [31]:
tools = [
    Tool(
        name="Calculator",
        func=llm_math_chain.run,
        description="useful for when you need to answer questions about math",
        return_direct=True,
    )
]

# 处理工具的Errors

In [42]:
from langchain.tools.base import ToolException

def _handle_error(error: ToolException) -> str:
    return(
        "The following errors occurred during tool execution:"
        + error.args[0]
        + "Please try another tool."
    )

In [43]:
def search_tool1(s: str):
    raise ToolException("The search tool1 is not available.")

def search_tool2(s: str):
    raise ToolException("The search tool2 is not available.")

search_tool3 = SerpAPIWrapper()



In [48]:
description = "useful for when you need to answer questions about current events.You should give priority to using it."
tools = [
    Tool.from_function(
        func=search_tool1,
        name="Search_tool1",
        description=description,
        handle_tool_error=True,
    ),
    Tool.from_function(
        func=search_tool2,
        name="Search_tool2",
        description=description,
        handle_tool_error=_handle_error,
    ),
    Tool.from_function(
        func=search_tool3.run,
        name="Search_tool3",
        description="useful for when you need to answer questions about current events",
    ),
]

agent = initialize_agent(
    tools,
    llm_azure,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)

In [46]:
agent.run("Who is Leo DiCaprio's girlfriend?")



> Entering new AgentExecutor chain...
This is a question about a current event, so I should prioritize using Search_tool1 or Search_tool2.
Action: Search_tool1
Action Input: "Leo DiCaprio girlfriend"
Observation: The search tool1 is not available.
Thought:I should try using Search_tool2 instead.
Action: Search_tool2
Action Input: "Leo DiCaprio girlfriend"
Observation: The following errors occurred during tool execution:The search tool2 is not available.Please try another tool.
Thought:I should try using Search_tool3 instead.
Action: Search_tool3
Action Input: "Leo DiCaprio girlfriend"
Observation: The actor is believed to have recently split from his girlfriend of five years, actor Camila Morrone, but has previously been romantically ...
Thought:Based on the observation, it seems that Leo DiCaprio's girlfriend is Camila Morrone, but they may have recently split up.
Final Answer: Leo DiCaprio's girlfriend is Camila Morrone, but their current relationship status is unclear.

> Finished

"Leo DiCaprio's girlfriend is Camila Morrone, but their current relationship status is unclear."